<a href="https://colab.research.google.com/github/Alf-caput/NFL2022_TacklePredictNN/blob/dev1/Practica_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NFL Big Data Bowl 2024

Kaggle lanza el concurso NFL Big Data Bowl 2024 sponsorizado por National Football League con un premio de $100,000.
https://www.kaggle.com/c/nfl-big-data-bowl-2024

## INTRODUCCIÓN

El fútbol americano es un deporte complejo donde la tarea principal de los defensores es derribar al portador del balón, mientras que este último busca avanzar en el campo. El siguiente proyecto tiene como objetivo crear métricas para valorar elementos clave en los placajes.

Se utiliza la información de las estadísticas Next Gen de la NFL, incluyendo datos de seguimiento de jugadores de las primeras nueve semanas de la temporada 2022. Estos datos abarcan la ubicación, velocidad y aceleración de los 22 jugadores en el campo, así como la posición del balón. Además, se incorporan datos de evaluación de jugadores de PFF y estadísticas avanzadas de la NFL, como puntos esperados y probabilidad de victoria.

La finalidad del proyecto es presentar los resultados a la NFL. Las métricas y análisis destacados podrían ser utilizados por los equipos de la NFL para evaluar a sus jugadores tanto en el ámbito ofensivo como defensivo.

## OBJETIVOS DEL CONCURSO

Se pide generar ideas prácticas, novedosas y ejecutables a partir de los datos de seguimiento de jugadores relacionados con los placajes. Algunos ejemplos incluyen, pero no se limitan a:

- Predicciones del tiempo, probabilidad y/o ubicación del derribo.
- Rango de derribo: ángulo de persecución, velocidad y aceleración, velocidad de cierre.
- Evaluación del jugador (por ejemplo, yardas salvadas, valor del derribo, derribos fallidos).
- Asignación de créditos (por ejemplo, un jugador realiza un derribo debido a otro jugador, bloqueos superados, área de influencia).
- Tipo de derribo (individual vs. en grupo, en campo abierto vs. en las trincheras, etc.).
- Roles y responsabilidades del equipo y los jugadores (marcar el límite, ocupar huecos, etc.).

Es importante destacar que la lista anterior no es exhaustiva y se anima a los participantes a ser creativos con sus propuestas.





